<a href="https://colab.research.google.com/github/Mirocan17/DSA-210-TERM-PROJECT/blob/main/notebooks/NBA_Tanking_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HYPOTHESIS TESTING
In this part, the statistical tests will be applied to evaluate the impacts of the strategy of the teams. Here we will examine 3 different hypothesis for our project:

H0 : There is no significance difference in the average wins of the bottom 3 teams between 2 eras. (before and after 2019)

H1 : There is significance difference in the average wins of the bottom 3 teams between 2 eras.

Tanking cannot be prevented %100 while there are generational prospects in the draft. 2019 reform decreased the chance of having the 1st overall pick in the NBA Draft for the worst team. After the reform in the 2019 to prevent teams from tanking by the NBA, here we will examine whether those teams started to lose less games intentionally or not.

H0 : There is no relationship between total losses and playoff success.

H1 : There is a significant correlation (positive or negative) between losses and success.

Here we will examine whether giving up short term successes give long term succeses during the period between (1994-2025)


In [14]:
import pandas as pd
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
pd.options.display.float_format = '{:.4f}'.format
try:
    print("🔬 RUNNING HYPOTHESIS TESTS (1 & 2 ONLY)...\n")
    try:
        df_stand = pd.read_csv('nba_standings_1994_2025_robust.csv')
        try:
            df_playoff = pd.read_csv('nba_playoff_outcomes_accurate_v3.csv')
        except:
            df_playoff = pd.read_csv('nba_playoff_outcomes_offline_final_v2.csv')
    except FileNotFoundError:
        print("ERROR: Files not found. Please ensure CSV files are uploaded.")
        raise
    print("🔹 HYPOTHESIS 1: The 'Zion Rule' Effect (Did the 2019 Reform Stop Tanking?)")
    print("   H0: The average wins of the bottom 3 teams Pre-2019 and Post-2019 are EQUAL.")
    print("   H1: The average wins of the bottom 3 teams are DIFFERENT (Reform had an impact).\n")
    bottom3_wins = []
    years = df_stand['Year'].unique()
    for year in years:
        if year == 2025: continue
        sorted_standings = df_stand[df_stand['Year'] == year].sort_values(by='W')
        bottom3 = sorted_standings.head(3)
        era = 'Pre-2019 (Old Odds)' if year < 2019 else 'Post-2019 (New Odds)'
        for w in bottom3['W']:
            bottom3_wins.append({'Year': year, 'Era': era, 'Wins': w})
    df_h1 = pd.DataFrame(bottom3_wins)
    group_old = df_h1[df_h1['Era'] == 'Pre-2019 (Old Odds)']['Wins']
    group_new = df_h1[df_h1['Era'] == 'Post-2019 (New Odds)']['Wins']
    t_stat, p_val = stats.ttest_ind(group_old, group_new, equal_var=False)
    print(f"   -> Pre-2019 Avg Wins (Bottom 3): {group_old.mean():.1f}")
    print(f"   -> Post-2019 Avg Wins (Bottom 3): {group_new.mean():.1f}")
    print(f"   -> P-Value: {p_val:.4f}")
    if p_val < 0.05:
        print("✅ RESULT: The difference is STATISTICALLY SIGNIFICANT! (Reject H0)")
        if group_new.mean() > group_old.mean():
            print("   Interpretation: The Zion Rule worked! The worst teams are winning significantly more games now.")
        else:
            print("   Interpretation: The rule backfired; teams are losing even more.")
    else:
        print("❌ RESULT: No Significant Difference. (Fail to reject H0)")
        print("   Interpretation: Changing the lottery odds did NOT statistically change tanking behavior.")
    print("\n" + "-"*50 + "\n")
    print("🔹 HYPOTHESIS 2: The 'Tanking Efficiency' Hypothesis")
    print("   Question: Is there a correlation between a team's Total Losses and their Playoff Success over 30 years?")
    print("   Test: Pearson Correlation (Total Losses vs. Total Success Score)\n")
    pain_df = df_stand.groupby('Team')['L'].sum().reset_index()
    score_map = {'Champion': 20, 'Finals': 10, 'Conf. Finals': 5, '2nd Round': 2, 'Semis': 2, '1st Round': 1, 'Missed Playoffs': 0}
    df_playoff['Success_Score'] = df_playoff['Playoff_Finish'].map(score_map).fillna(0)
    gain_df = df_playoff.groupby('Team')['Success_Score'].sum().reset_index()
    merged_df = pd.merge(pain_df, gain_df, on='Team')
    corr_coef, p_val_corr = stats.pearsonr(merged_df['L'], merged_df['Success_Score'])
    print(f"   -> Correlation Coefficient (r): {corr_coef:.4f}")
    print(f"   -> P-Value: {p_val_corr:.4f}")
    if p_val_corr < 0.05:
        print("✅ RESULT: There is a STATISTICALLY SIGNIFICANT correlation.")
        if corr_coef > 0:
            print("   Direction: POSITIVE (More Losses = More Success). Tanking works!")
        else:
            print("   Direction: NEGATIVE (More Losses = Less Success). Tanking is detrimental!")
    else:
        print("❌ RESULT: No Significant Correlation. (Fail to reject H0)")
        print("   Interpretation: Accumulating losses does NOT guarantee future success. Organizational culture matters more than draft position.")
except Exception as e:
    print(f"Error occurred: {e}")

🔬 RUNNING HYPOTHESIS TESTS (1 & 2 ONLY)...

🔹 HYPOTHESIS 1: The 'Zion Rule' Effect (Did the 2019 Reform Stop Tanking?)
   H0: The average wins of the bottom 3 teams Pre-2019 and Post-2019 are EQUAL.
   H1: The average wins of the bottom 3 teams are DIFFERENT (Reform had an impact).

   -> Pre-2019 Avg Wins (Bottom 3): 18.7
   -> Post-2019 Avg Wins (Bottom 3): 19.0
   -> P-Value: 0.7299
❌ RESULT: No Significant Difference. (Fail to reject H0)
   Interpretation: Changing the lottery odds did NOT statistically change tanking behavior.

--------------------------------------------------

🔹 HYPOTHESIS 2: The 'Tanking Efficiency' Hypothesis
   Question: Is there a correlation between a team's Total Losses and their Playoff Success over 30 years?
   Test: Pearson Correlation (Total Losses vs. Total Success Score)

   -> Correlation Coefficient (r): 0.4208
   -> P-Value: 0.0095
✅ RESULT: There is a STATISTICALLY SIGNIFICANT correlation.
   Direction: POSITIVE (More Losses = More Success). Tank

H0 : The win differential between tanking teams and mediocre teams after 5 years is greater than or equal to 7 wins.

H1 : The win differential is less than 7 wins.

In this part we aim to analyse whether it is logical to tank heavy when the team actually can be middle table team. Will it really affect the future of the team?

In [12]:
import pandas as pd
from scipy import stats
import numpy as np
pd.options.display.float_format = '{:.4f}'.format
print("🔬ONE-TAILED TEST\n")
df_stand = pd.read_csv('nba_standings_1994_2025_robust.csv')
print("🔹 HYPOTHESIS 3: The 'Magic Leap' Superiority Test")
print("   Question: Do heavy tankers gain a massive advantage (>= 7 Wins) over mediocre teams after 5 years?")
tanking_future_wins = []
mediocre_future_wins = []
for year in range(1994, 2020):
    season_data = df_stand[df_stand['Year'] == year]
    future_data = df_stand[df_stand['Year'] == year + 5]
    if season_data.empty or future_data.empty: continue
    merged = pd.merge(season_data[['Team', 'W']], future_data[['Team', 'W']], on='Team', suffixes=('_Current', '_Future'))
    tankers = merged[merged['W_Current'] < 20]['W_Future']
    mediocre = merged[(merged['W_Current'] >= 30) & (merged['W_Current'] <= 45)]['W_Future']
    tanking_future_wins.extend(tankers.tolist())
    mediocre_future_wins.extend(mediocre.tolist())
mean_tank = np.mean(tanking_future_wins)
mean_med = np.mean(mediocre_future_wins)
std_tank = np.std(tanking_future_wins, ddof=1)
std_med = np.std(mediocre_future_wins, ddof=1)
n1 = len(tanking_future_wins)
n2 = len(mediocre_future_wins)
target_diff = 7
se_diff = np.sqrt((std_tank**2 / n1) + (std_med**2 / n2))
actual_diff = mean_tank - mean_med
t_stat = (actual_diff - target_diff) / se_diff
df = n1 + n2 - 2
p_val = stats.t.cdf(t_stat, df)
print(f"   -> Avg Wins (Tankers): {mean_tank:.1f}")
print(f"   -> Avg Wins (Mediocre): {mean_med:.1f}")
print(f"   -> Actual Difference: {actual_diff:.1f} Wins")
print(f"   -> Target Difference (H0 bound): >= {target_diff:.1f} Wins")
print(f"   -> T-Statistic: {t_stat:.4f}")
print(f"   -> P-Value (One-Tailed): {p_val:.10f}")
if p_val < 0.05:
    print("\n✅ RESULT: Reject H0. The difference is SIGNIFICANTLY LESS than 7 wins.")
    print("   Interpretation: We can scientifically prove that tanking does NOT provide a +7 win 'Super Boost' within 5 years.")
    print("   The strategy fails to deliver the promised massive returns.")
else:
    print("\n❌ RESULT: Fail to Reject H0.")
    print("   Interpretation: We cannot rule out that tanking provides a +7 win advantage.")

🔬ONE-TAILED TEST

🔹 HYPOTHESIS 3: The 'Magic Leap' Superiority Test
   Question: Do heavy tankers gain a massive advantage (>= 7 Wins) over mediocre teams after 5 years?
   -> Avg Wins (Tankers): 39.0
   -> Avg Wins (Mediocre): 39.5
   -> Actual Difference: -0.5 Wins
   -> Target Difference (H0 bound): >= 7.0 Wins
   -> T-Statistic: -3.4878
   -> P-Value (One-Tailed): 0.0002764980

✅ RESULT: Reject H0. The difference is SIGNIFICANTLY LESS than 7 wins.
   Interpretation: We can scientifically prove that tanking does NOT provide a +7 win 'Super Boost' within 5 years.
   The strategy fails to deliver the promised massive returns.
